# Windows Process

In this section, we briefly cover the expected processes during boot and startup of a Windows system. Why is this important? Simply so we know what is normal, and what is (possibly) suspicious. As with all investigations, the analyst must have the understanding and context to say what is worth investigating and reporting on. Hopefully this gives some guidance to you. If you want the TLDR; I've included the simplified flow diagram below. If you want to modify it for your purpose, the Draw.IO file is also available in the root directory / drawings.<br>

![WindowsProcesses.png](../../Images/Goals/DFIR/WindowsProcesses.png)<br>

## Processes

Processes are simply programs running in the background of the system. Well, kinda, but it's easier to think of it this way for now. Processes can be seen in Task Manager on modern Windows versions, but a far more common tool is "Process Explorer", which forms a part of SysInternals. I don't dive into this here.

### System
The system process is the first launched after boot. It's role is to call all other processes used for booting the OS. As such it shouldn't have any parent processes and it is always process ID 4. You may be asking why the first one is #4, well I was wondering that too. Simply it's because process IDs are divisible by 4 due to a property of the process handles, the lower 2 bits of the ID are used for tags. Therefore the first PID after 0 is 4. Bit of trivia for you. Anyways....<br>
<br>

Expected Properties:<br>

> Process Count: One instance only<br>
> Parent Processes: No Parent process (there may be an exception for the idle process, not sure why) <br>
> User: Local System <br>
> Start Time: At Boot <br>
> Path: C:\Windows\System32\ntoskrnl.exe <br>
> Special Information: ALWAYS PID 4<br>

### SMSS
The Session Manager Sub System (SMSS) is also known as the Windows Session Manger. Don't confuse it with SSMS (Sql Server Management Studio) - I know I keep mixing them up... It is launched by the 'system' process and has a seperate instance for each 'session' on the machine. Session 0 is always reserved for the system. If you aren't sure what a 'session' is, I always relate it back to RDP. On a server, you can have multiple 'active' users logged into a system at once. Under the hood, the RDP session just launches a new session to be used. You can also use this to kill connections (worst case scenario) but you'll learn how important Session 0 is if you do this accidentally....<br>

Expected Properties:<br>

> Process Count: Multiple Possible <br>
> Parent Processes: System (PID4) <br>
> User: Local System <br>
> Start Time: At Boot for Session 0, login time for others <br>
> Path: C:\Windows\System32\smss.exe <br>
> Special Information: Launches SRSS and WinInit then terminates - shouldn't be running normally <br>

### CSRSS
The Client Server Runtime proceSS (yes, it uses the end of process). It is launched by SMSS and inherits its session number. As SMSS terminates after launching it's child processes, CSRSS shouldn't have a parent process known. Again, Session 0 is reserved for the OS and should be running.<br>

Expected Properties:<br>

> Process Count: Multiple Possible <br>
> Parent Processes: SMSS (should show no parent or parent not found) <br>
> User: Local System <br>
> Start Time: At Boot for Session 0, login time for others <br>
> Path: C:\Windows\System32\csrss.exe <br>
> Special Information: Inherets session number from SMSS<br>

### WinInit
The Windows Initalisation Process is another one launched by SMSS so the properties are expected to be about the same.
Expected Properties:<br>

> Process Count: Multiple Possible <br>
> Parent Processes: SMSS (should show no parent or parent not found) <br>
> User: Local System <br>
> Start Time: At Boot for Session 0, login time for others <br>
> Path: C:\Windows\System32\wininit.exe <br>
> Special Information: Should match the launch time of CSRSS for the same session<br>

